### Logic based solution for finding truly lapsed and truly new members

In [1]:
!pip install fuzzywuzzy
!pip install openpyxl
!pip install rapidfuzz

In [4]:
act=pd.read_excel('fuzzy_data/Lapsed paid members Dec 2024 .xlsx')
act.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country
0,640199,ANASTASIIA KOVTUN,995402520,ANASTASIIA KOVTUN,"7 Gagarina st, Mykolaivka",Dnipro,UA,UKR
1,654753,Murazi Soinogli,552800003,MURAZI SOINOGLI,S.ehmedov Ev 9q Baku,Baku,AJ,AZB
2,294594,KODY TODD CLARK,3072482063,KODY TODD CLARK,876 EL CLARK LN PO Box 15,Smoot,WY,USA
3,886721,KYLE JACOB CAMPBELL,9713133773,KYLE JACOB CAMPBELL,115 Prospect St,Bennington,VT,USA
4,646045,ASTRID VIRGINIA GONZALEZ GUERRERO,54140700,ASTRID VIRGINIA GONZALEZ GUERR,"Calle Mariscal, 5-26, zona 11",Guatemala,GT,GTM


In [5]:
act.tail()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country
413708,1265516936516110080,Levi Hoyle,NaN,G AND L MOTORS LLC,101 SKYLINE DR # 1,ARLINGTON,WI,USA
413709,838919,MARK LUND,9207872277,LUND NORTHWOODS AUTO &REPAIR,N2863 STATE RD 73,Wautoma,WI,USA
413710,837617,OFELIA GARCIA ANCHONDO,6082097825,OFELIA GARCIA ANCHONDO,121 East Main street,Madison,WI,USA
413711,367538,JASON FARRELL,9205735750,Farrell Performance LLC,2300 S. Washburn St.,Oshkosh,WI,USA
413712,351470,STEVEN CONNELL,7158947227,STEVEN CONNELL,19372 county highway Oo,Chippewa falls,WI,USA


# 1

In [18]:
import pandas as pd
import numpy as np
from numpy import mean,std
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,Normalizer,MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics


import plotly.express as px


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#act=pd.read_csv('anant_17k.csv').rename(columns={'mber_name': 'mbr_name'})

#act=pd.read_csv('Lapsed paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/Lapsed paid members Dec 2024 (1).xlsx')
#lap=pd.read_csv('New Premier Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/New Premiers Dec 2024.xlsx').dropna()
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,ELKHORN,WI,USA,DULI USA,797 E GENEVA # 18,DULI USA 797 E GENEVA # 18,USA WI ELKHORN 797 E GENEVA # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,KRENAR DULI DULI USA,1
1,898971,BRIAN DOUGLAS STRONG,2147971031,BRIAN DOUGLAS STRONG,704 Alysa Lane,WAXAHACHIE,TX,USA,BRIAN DOUGLAS STRONG,704 ALYSA LANE,BRIAN DOUGLAS STRONG 704 ALYSA LANE,USA TX WAXAHACHIE 704 ALYSA LANE,BRIAN DOUGLAS STRONG BRIAN DOUGLAS STRONG USA ...,BRIAN DOUGLAS STRONG BRIAN DOUGLAS STRONG,1
2,81054,LUIS ESTUARDO MARTINEZ BARILLAS,41494333,PREMIUM SURTIAUTOS,"EJE 1 CASA20 JACARANDAS PLANES DE BARCENAS, Z3...",CUIDAD DE GUATEMALA,GT,GTM,PREMIUM SURTIS,"EJE 1 CASA20 JACARANDAS PLANES DE BARCENAS, Z3...",PREMIUM SURTIS EJE 1 CASA20 JACARANDAS PLANES ...,GTM GT CUIDAD DE GUATEMALA EJE 1 CASA20 JACARA...,LUIS ESTUARDO MARTINEZ BARILLAS PREMIUM SURTIS...,LUIS ESTUARDO MARTINEZ BARILLAS PREMIUM SURTIS,1
3,22534,RAYMOND J WILLEMS,95221370,RAYMOND J WILLEMS,DKT JONKHEERSTRAAT 3,WILLEMSTAD,CC,CUR,RAYMOND J WILLEMS,DKT JONKHEERRAAT 3,RAYMOND J WILLEMS DKT JONKHEERRAAT 3,CUR CC WILLEMSTAD DKT JONKHEERRAAT 3,RAYMOND J WILLEMS RAYMOND J WILLEMS CUR CC WIL...,RAYMOND J WILLEMS RAYMOND J WILLEMS,1
4,915633,MUHAMMAD IMRAN ABBAALI,558893907,SHALIMAR MOTORS FZCO,85931 AZ02P195,DUBAI,UE,ARE,SHALIMAR FZCO,85931 AZ02P195,SHALIMAR FZCO 85931 AZ02P195,ARE UE DUBAI 85931 AZ02P195,MUHAMMAD IMRAN ABBAALI SHALIMAR FZCO ARE UE D...,MUHAMMAD IMRAN ABBAALI SHALIMAR FZCO,1


In [19]:

lap.tail(2)
act.tail(2)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
1782,116070,MARK PAUL CIMOLI,2098900779,C6 MOTORSPORTS LLC,4200 DUNN ROAD,VALLEY SPRINGS,CA,USA,C6 PORTS,4200 DUNN,C6 PORTS 4200 DUNN,USA CA VALLEY SPRINGS 4200 DUNN,MARK PAUL CIMOLI C6 PORTS USA CA VALLEY SPRIN...,MARK PAUL CIMOLI C6 PORTS,1
1783,909409,JOSE DANIEL HERNANDEZ CUEVAS,2225192588,JOSE DANIEL HERNANDEZ CUEVAS,PROFESORA RAMONA CHACON 533 HORIZONTES DEL SUR,JUAREZ,CH,MEX,JOSE DANIEL HERNANDEZ CUEVAS,PROFESORA RAMONA CHACON 533 HORIZONTES DEL SUR,JOSE DANIEL HERNANDEZ CUEVAS PROFESORA RAMONA ...,MEX CH JUAREZ PROFESORA RAMONA CHACON 533 HORI...,JOSE DANIEL HERNANDEZ CUEVAS JOSE DANIEL HERNA...,JOSE DANIEL HERNANDEZ CUEVAS JOSE DANIEL HERNA...,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
330083,418130,TIMOTHY THOMAS,2404165075,ROYAL-T AUTO GROUP LLC,101 Skyline Drive #1 W421,ARLINGTON,WI,USA,ROYAL-T GROUP,101 SKYLINE #1 W421,ROYAL-T GROUP 101 SKYLINE #1 W421,USA WI ARLINGTON 101 SKYLINE #1 W421,TIMOTHY THOMAS ROYAL-T GROUP USA WI ARLINGTO...,TIMOTHY THOMAS ROYAL-T GROUP,0
330084,275610,JEROME PACE SR,7737427786,TOKYO AUTO GROUP LLC,101 SKYLINE DR # 1 W431,ARLINGTON,WI,USA,TOKYO GROUP,101 SKYLINE # 1 W431,TOKYO GROUP 101 SKYLINE # 1 W431,USA WI ARLINGTON 101 SKYLINE # 1 W431,JEROME PACE SR TOKYO GROUP USA WI ARLINGTON ...,JEROME PACE SR TOKYO GROUP,0


In [20]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('fuzzy_data/New Premiers Dec 2024 Lapsed Paid Members Dec 2024 Fuzzy.csv')

--- 3593.1571791172028 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,604335,RICHARD HANSEN,4802008722,USA AUTO INC,525 W MAIN ST,RICHARD HANSEN USA USA AZ MESA 525 W MAIN,0,44.444444,48.484848,44.444444,32.000000,100.0,hier 4
1,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,887352,ALEJANDRO HUESO,8588670349,USA AUTO SALES,2660 Imperial Ave.,ALEJANDRO HUESO USA USA CA SAN DIEGO 2660 IM...,0,39.130435,47.058824,40.625000,30.769231,100.0,hier 4
2,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,551628,SHIH LIANG HUANG,8585853434,CARUSA LLC,15859 EDNA PL STE 112,SHIH LIANG HUANG USA USA CA IRWINDALE 15859 E...,0,45.360825,45.714286,52.941176,29.629630,100.0,hier 4
3,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,897523,MUHAMMAD AKHTAR,5303152276,USA AUTO SALE,4992 OLIVEHURST AVE OLIVEHURST,MUHAMMAD AKHTAR USA USA CA OLIVEHURST 4992 O...,0,37.362637,41.176471,38.095238,23.076923,100.0,hier 4
4,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,312334,RICHARD YNSANA GIAMMARINO,4042633914,USA MOTORS,1234 beaver ruin rd ste 203,RICHARD YNSANA GIAMMARINO USA USA GA LAWRENCE...,0,41.818182,40.909091,52.777778,27.777778,100.0,hier 4


# 2


In [21]:

#act=pd.read_csv('active paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/active paid members Dec 2024.xlsx')
#lap=pd.read_csv('New Premier Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/New Premiers Dec 2024.xlsx').dropna()
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,ELKHORN,WI,USA,DULI USA,797 E GENEVA # 18,DULI USA 797 E GENEVA # 18,USA WI ELKHORN 797 E GENEVA # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,KRENAR DULI DULI USA,1
1,898971,BRIAN DOUGLAS STRONG,2147971031,BRIAN DOUGLAS STRONG,704 Alysa Lane,WAXAHACHIE,TX,USA,BRIAN DOUGLAS STRONG,704 ALYSA LANE,BRIAN DOUGLAS STRONG 704 ALYSA LANE,USA TX WAXAHACHIE 704 ALYSA LANE,BRIAN DOUGLAS STRONG BRIAN DOUGLAS STRONG USA ...,BRIAN DOUGLAS STRONG BRIAN DOUGLAS STRONG,1
2,81054,LUIS ESTUARDO MARTINEZ BARILLAS,41494333,PREMIUM SURTIAUTOS,"EJE 1 CASA20 JACARANDAS PLANES DE BARCENAS, Z3...",CUIDAD DE GUATEMALA,GT,GTM,PREMIUM SURTIS,"EJE 1 CASA20 JACARANDAS PLANES DE BARCENAS, Z3...",PREMIUM SURTIS EJE 1 CASA20 JACARANDAS PLANES ...,GTM GT CUIDAD DE GUATEMALA EJE 1 CASA20 JACARA...,LUIS ESTUARDO MARTINEZ BARILLAS PREMIUM SURTIS...,LUIS ESTUARDO MARTINEZ BARILLAS PREMIUM SURTIS,1
3,22534,RAYMOND J WILLEMS,95221370,RAYMOND J WILLEMS,DKT JONKHEERSTRAAT 3,WILLEMSTAD,CC,CUR,RAYMOND J WILLEMS,DKT JONKHEERRAAT 3,RAYMOND J WILLEMS DKT JONKHEERRAAT 3,CUR CC WILLEMSTAD DKT JONKHEERRAAT 3,RAYMOND J WILLEMS RAYMOND J WILLEMS CUR CC WIL...,RAYMOND J WILLEMS RAYMOND J WILLEMS,1
4,915633,MUHAMMAD IMRAN ABBAALI,558893907,SHALIMAR MOTORS FZCO,85931 AZ02P195,DUBAI,UE,ARE,SHALIMAR FZCO,85931 AZ02P195,SHALIMAR FZCO 85931 AZ02P195,ARE UE DUBAI 85931 AZ02P195,MUHAMMAD IMRAN ABBAALI SHALIMAR FZCO ARE UE D...,MUHAMMAD IMRAN ABBAALI SHALIMAR FZCO,1


In [22]:

lap.tail(2)
act.tail(2)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
1782,116070,MARK PAUL CIMOLI,2098900779,C6 MOTORSPORTS LLC,4200 DUNN ROAD,VALLEY SPRINGS,CA,USA,C6 PORTS,4200 DUNN,C6 PORTS 4200 DUNN,USA CA VALLEY SPRINGS 4200 DUNN,MARK PAUL CIMOLI C6 PORTS USA CA VALLEY SPRIN...,MARK PAUL CIMOLI C6 PORTS,1
1783,909409,JOSE DANIEL HERNANDEZ CUEVAS,2225192588,JOSE DANIEL HERNANDEZ CUEVAS,PROFESORA RAMONA CHACON 533 HORIZONTES DEL SUR,JUAREZ,CH,MEX,JOSE DANIEL HERNANDEZ CUEVAS,PROFESORA RAMONA CHACON 533 HORIZONTES DEL SUR,JOSE DANIEL HERNANDEZ CUEVAS PROFESORA RAMONA ...,MEX CH JUAREZ PROFESORA RAMONA CHACON 533 HORI...,JOSE DANIEL HERNANDEZ CUEVAS JOSE DANIEL HERNA...,JOSE DANIEL HERNANDEZ CUEVAS JOSE DANIEL HERNA...,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
315125,933470,BRIAN HABLE,6513298045,BRIAN HABLE,631 Highway 65,ROBERTS,WI,USA,BRIAN HABLE,631 65,BRIAN HABLE 631 65,USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE,0
315126,944029,FREDY A CORRO RUIZ,2624244134,MONARCAS AUTO LLC,5610 W NATIONAL AVE,WEST ALLIS,WI,USA,MONARCAS,5610 W NATIONAL,MONARCAS 5610 W NATIONAL,USA WI WEST ALLIS 5610 W NATIONAL,FREDY A CORRO RUIZ MONARCAS USA WI WEST ALL...,FREDY A CORRO RUIZ MONARCAS,0


In [23]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('New Premiers Dec 2024 Active Paid Members Dec 2024 Fuzzy.csv')

--- 3422.27002620697 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,369475,JUAN ESCOBAR REGALADO,98434741,USA CAR,SUR-ESTE URBANO TEPEACA CALLE 4 AVE 12 LOCAL 213,JUAN ESCOBAR REGALADO USA HND HN SAN PEDRO SU...,0,37.313433,45.000000,35.416667,31.250000,100.0,hier 4
1,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,337860,ARTUSH AMARYAN,43666006,USA MOTORS LLC,"36 STR ARMENAKYAN, NORK MARASH",ARTUSH AMARYAN USA ARM AM YEREVAN 36 R ARMEN...,0,35.849057,48.484848,34.666667,32.000000,100.0,hier 4
2,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,312900,ARAM HARUTYUNYAN,43666006,USA MOTORS LLC,"36 STR ARMENAKYAN, NORK MARASH",ARAM HARUTYUNYAN USA ARM AM YEREVAN 36 R ARM...,0,37.037037,45.714286,34.666667,29.629630,100.0,hier 4
3,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,794982,FARES A HAYEK,8608162545,USA AUTO SALES LLC,147 New Britain Road,FARES A HAYEK USA USA CT BERLIN 147 NEW BRI...,0,43.678161,50.000000,49.180328,33.333333,100.0,hier 4
4,910951,KRENAR DULI,8723148751,AUTO DULI USA INC,797 E GENEVA ST # 18,KRENAR DULI DULI USA USA WI ELKHORN 797 E GE...,1,597437,CONSTANTINE Y SHEVCHENKO,6783999150,USACARSALES,2759 DELK ROAD SUITE 1971,CONSTANTINE Y SHEVCHENKO USA USA GA MARIETTA 2...,0,45.714286,37.209302,52.941176,22.857143,100.0,hier 4


# 3


In [27]:

#act=pd.read_csv('active paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/active paid members Dec 2024.xlsx')
#lap=pd.read_csv('Lapsed Premier Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/Lapsed Premier Dec 2024.xlsx').dropna().rename(columns={'mbr_nbr_1': 'mbr_nbr'})
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,339024,MARISABEL D VILLALOBOS TELLO,7865311950,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY AVE,BOLINGBROOK,IL,USA,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY,MARISABEL D VILLALOBOS TELLO 255 N ASHBURY,USA IL BOLINGBROOK 255 N ASHBURY,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,1
1,251991,DAVID RICARDO FERNANDEZ SEGOVIA,6125177319,DAVID RICARDO FERNANDEZ SEGOVI,219 Darrick Rd,CHASKA,MN,USA,DAVID RIDO FERNANDEZ SEGOVI,219 DARRICK,DAVID RIDO FERNANDEZ SEGOVI 219 DARRICK,USA MN CHASKA 219 DARRICK,DAVID RICARDO FERNANDEZ SEGOVIA DAVID RIDO FER...,DAVID RICARDO FERNANDEZ SEGOVIA DAVID RIDO FER...,1
2,889919,JORGE CARDONA,6828886785,JORGE CARDONA,8833 Soy Seed Trail,FORT WORTH,TX,USA,JORGE DONA,8833 SOY SEED TRAIL,JORGE DONA 8833 SOY SEED TRAIL,USA TX FORT WORTH 8833 SOY SEED TRAIL,JORGE CARDONA JORGE DONA USA TX FORT WORTH 883...,JORGE CARDONA JORGE DONA,1
3,861159,PAULO DOS SANTOS,9047185065,PAULO DOS SANTOS,3425 DEVILWOOD ST,MIDDLEBURG,FL,USA,PAULO DOS SANTOS,3425 DEVILWOOD,PAULO DOS SANTOS 3425 DEVILWOOD,USA FL MIDDLEBURG 3425 DEVILWOOD,PAULO DOS SANTOS PAULO DOS SANTOS USA FL MIDDL...,PAULO DOS SANTOS PAULO DOS SANTOS,1
4,442336,MOUSA JAMIL ABUNASIR,64737582,MOUSA JAMIL ABUNASER & PARTNER,AL YARMOUK ST/AL WEHDAT MIDDLE EAST CIRCLE,AMMAN,JN,JOR,MOUSA JAMIL ABUNASER & NER,AL YARMOUK /AL WEHDAT MIDDLE EA CIRCLE,MOUSA JAMIL ABUNASER & NER AL YARMOUK /AL WEHD...,JOR JN AMMAN AL YARMOUK /AL WEHDAT MIDDLE EA C...,MOUSA JAMIL ABUNASIR MOUSA JAMIL ABUNASER & N...,MOUSA JAMIL ABUNASIR MOUSA JAMIL ABUNASER & NER,1


In [28]:

lap.tail(2)
act.tail(2)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
2783,83144,DRAZEN KATINIC,8593075876,AIR RUSH HVAC LLC,167 Ridgewood Drive,ALEXANDRIA,KY,USA,AIR RUSH HVAC,167 RIDGEWOOD,AIR RUSH HVAC 167 RIDGEWOOD,USA KY ALEXANDRIA 167 RIDGEWOOD,DRAZEN KATINIC AIR RUSH HVAC USA KY ALEXANDRI...,DRAZEN KATINIC AIR RUSH HVAC,1
2784,91184,IVAN KUTSAR,3609105903,IVAN KUTSAR,4719 se 62nd Ave,PORTLAND,OR,USA,IVAN KUTSAR,4719 SE 62ND,IVAN KUTSAR 4719 SE 62ND,USA OR PORTLAND 4719 SE 62ND,IVAN KUTSAR IVAN KUTSAR USA OR PORTLAND 4719 S...,IVAN KUTSAR IVAN KUTSAR,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
315125,933470,BRIAN HABLE,6513298045,BRIAN HABLE,631 Highway 65,ROBERTS,WI,USA,BRIAN HABLE,631 65,BRIAN HABLE 631 65,USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE,0
315126,944029,FREDY A CORRO RUIZ,2624244134,MONARCAS AUTO LLC,5610 W NATIONAL AVE,WEST ALLIS,WI,USA,MONARCAS,5610 W NATIONAL,MONARCAS 5610 W NATIONAL,USA WI WEST ALLIS 5610 W NATIONAL,FREDY A CORRO RUIZ MONARCAS USA WI WEST ALL...,FREDY A CORRO RUIZ MONARCAS,0


In [29]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('Lapsed Premiers Dec 2024 Active Paid Members Dec 2024 Fuzzy.csv')

--- 5654.314939022064 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,339024,MARISABEL D VILLALOBOS TELLO,7865311950,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY AVE,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,1,77519,ANA VILLALOBOS,9096759378,A&0 UNITED VICTORY,222 East Foothill Boulevard 76,ANA VILLALOBOS A&0 UNITED VICTORY USA CA POMON...,0,50.370370,59.016393,44.444444,83.333333,34.782609,hier 4
1,339024,MARISABEL D VILLALOBOS TELLO,7865311950,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY AVE,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,1,296748,CARLOS VILLALOBOS,7123558905,CARLOS VILLALOBOS,1525 25th ave,CARLOS VILLALOBOS LOS VILLALOBOS USA IA COUNCI...,0,57.894737,69.387755,50.793651,74.074074,83.333333,hier 4
2,339024,MARISABEL D VILLALOBOS TELLO,7865311950,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY AVE,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,1,216411,CARLOS DANIEL GARCIA GOMEZ,7087708407,CARLOS DANIEL GARCIA GOMEZ,261 vista dr,CARLOS DANIEL GARCIA GOMEZ LOS DANIEL GARCIA G...,0,64.406780,41.379310,81.818182,37.037037,47.058824,hier 4
3,339024,MARISABEL D VILLALOBOS TELLO,7865311950,MARISABEL D VILLALOBOS TELLO,255 N ASHBURY AVE,MARISABEL D VILLALOBOS TELLO MARISABEL D VILLA...,1,831166,ANTONIO MCCUMBER,1803426872,VILLALOBOS AUTO SALES INC.,306 Edgefield Rd,ANTONIO MCCUMBER VILLALOBOS . USA SC NORTH A...,0,49.600000,56.140351,39.393939,27.272727,90.909091,hier 4
4,251991,DAVID RICARDO FERNANDEZ SEGOVIA,6125177319,DAVID RICARDO FERNANDEZ SEGOVI,219 Darrick Rd,DAVID RICARDO FERNANDEZ SEGOVIA DAVID RIDO FER...,1,895510,RICARDO HIGGS,9546693372,RICARDO AUTO SALES,HIGH ROCK,RICARDO HIGGS RIDO BHS BS GRAND BAHAMA HIGH ...,0,47.863248,80.000000,29.629630,70.000000,100.000000,hier 4


In [ ]:
from rapidfuzz import fuzz
import pandas as pd
from joblib import Parallel, delayed
import time
# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
print ("start")
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif ((name_comp_score>70) |  (add_score>70)):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 80) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score', 'rank'])


print("--- %s seconds ---" % (time.time() - start_time))
mat.head()

In [ ]:
mat.to_csv('50_200_match.csv')

In [ ]:
mat=pd.DataFrame(columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'fuzzy score', 'rank'])
x

In [ ]:
# Convert the result to a DataFrame
mat = pd.DataFrame(mat, columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'fuzzy score', 'rank'])

print("--- %s seconds ---" % (time.time() - start_time))
mat.head()


# Dave's Request

In [3]:
import pandas as pd
import numpy as np
from numpy import mean,std
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,Normalizer,MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics


import plotly.express as px


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#act=pd.read_csv('anant_17k.csv').rename(columns={'mber_name': 'mbr_name'})

#act=pd.read_csv('Lapsed paid members Sep 2024.csv')
act=pd.read_excel('Copy of SalvageBuyers.xlsx').rename(columns={'Id': 'mbr_nbr'})
lap=pd.read_csv('MBR_active_paid_members.csv').dropna()
#lap=pd.read_excel('New Premiers Nov 2024.xlsx').dropna()
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')
act['mbr_country']='USA'
act['mbr_phone1'] = act['mbr_phone1'].str.replace(r'^(?:\(\+\d+\))|\D', '', regex=True)
act['mbr_phone1'] = act['mbr_phone1'].astype(float)

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,...,name_company,lapsed_flag,FirstName,LastName,UserName,mbr_email,Street2,ZipPostal,LastLoginDate,LastActivityDate
0,96017.0,TAHA ALGBURI,9.045846e+09,DURRAT ALEHAB FOR VEHICLE,BEHIND ALBUSTANJI MOTORS,ZARQA,JN,JOR,DURRAT ALEHAB FOR VEHICLE,BEHIND ALBUANJI MOTORS,...,TAHA ALGBURI DURRAT ALEHAB FOR VEHICLE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,69830.0,CORBIN CRAWFORD,6.053808e+09,CORBIN CARS,506 Main St,HURLEY,SD,USA,CORBIN S,506 MAIN,...,CORBIN CRAWFORD CORBIN S,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,326657.0,SPENSER LAWRENCE BUCK,9.712353e+09,SPENSER LAWRENCE BUCK,"8130 Williwa Ave, Unit A Anchorage",ANCHORAGE,AK,USA,SPENSER LAWRENCE BUCK,"8130 WILLIWA , A ANCHORAGE",...,SPENSER LAWRENCE BUCK SPENSER LAWRENCE BUCK,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,157859.0,EMMANUEL TOVIAS IBARRA,8.341544e+09,EMMANUEL TOVIAS IBARRA,Juan B Tijerina 203 D,CIUDAD VICTORIA,TM,MEX,EMMANUEL TOVIAS IBARRA,JUAN B TIJERINA 203 D,...,EMMANUEL TOVIAS IBARRA EMMANUEL TOVIAS IBARRA,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,92729.0,CRAIG TOMBS,7.769774e+09,CRAIG TOMBS,6 brodawel Penywaun,ABERDARE,GB,GBR,CRAIG TOMBS,6 BRODAWEL PENYWAUN,...,CRAIG TOMBS CRAIG TOMBS,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [4]:
lap.head()
lap.tail(5)
act.head(5)
act.tail(5)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,96017.0,TAHA ALGBURI,9.045846e+09,DURRAT ALEHAB FOR VEHICLE,BEHIND ALBUSTANJI MOTORS,ZARQA,JN,JOR,DURRAT ALEHAB FOR VEHICLE,BEHIND ALBUANJI MOTORS,DURRAT ALEHAB FOR VEHICLE BEHIND ALBUANJI MOTORS,JOR JN ZARQA BEHIND ALBUANJI MOTORS,TAHA ALGBURI DURRAT ALEHAB FOR VEHICLE JOR JN ...,TAHA ALGBURI DURRAT ALEHAB FOR VEHICLE,1
1,69830.0,CORBIN CRAWFORD,6.053808e+09,CORBIN CARS,506 Main St,HURLEY,SD,USA,CORBIN S,506 MAIN,CORBIN S 506 MAIN,USA SD HURLEY 506 MAIN,CORBIN CRAWFORD CORBIN S USA SD HURLEY 506 MAIN,CORBIN CRAWFORD CORBIN S,1
2,326657.0,SPENSER LAWRENCE BUCK,9.712353e+09,SPENSER LAWRENCE BUCK,"8130 Williwa Ave, Unit A Anchorage",ANCHORAGE,AK,USA,SPENSER LAWRENCE BUCK,"8130 WILLIWA , A ANCHORAGE","SPENSER LAWRENCE BUCK 8130 WILLIWA , A ANCHORAGE","USA AK ANCHORAGE 8130 WILLIWA , A ANCHORAGE",SPENSER LAWRENCE BUCK SPENSER LAWRENCE BUCK US...,SPENSER LAWRENCE BUCK SPENSER LAWRENCE BUCK,1
3,157859.0,EMMANUEL TOVIAS IBARRA,8.341544e+09,EMMANUEL TOVIAS IBARRA,Juan B Tijerina 203 D,CIUDAD VICTORIA,TM,MEX,EMMANUEL TOVIAS IBARRA,JUAN B TIJERINA 203 D,EMMANUEL TOVIAS IBARRA JUAN B TIJERINA 203 D,MEX TM CIUDAD VICTORIA JUAN B TIJERINA 203 D,EMMANUEL TOVIAS IBARRA EMMANUEL TOVIAS IBARRA ...,EMMANUEL TOVIAS IBARRA EMMANUEL TOVIAS IBARRA,1
4,92729.0,CRAIG TOMBS,7.769774e+09,CRAIG TOMBS,6 brodawel Penywaun,ABERDARE,GB,GBR,CRAIG TOMBS,6 BRODAWEL PENYWAUN,CRAIG TOMBS 6 BRODAWEL PENYWAUN,GBR GB ABERDARE 6 BRODAWEL PENYWAUN,CRAIG TOMBS CRAIG TOMBS GBR GB ABERDARE 6 BROD...,CRAIG TOMBS CRAIG TOMBS,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
314421,621332.0,TRAVIS GREGORY DAVENPORT,6.085765e+09,D PORT EQUIPMENT LLC,4320 HOEPKER RD,MADISON,WI,USA,D PORT EQUIPMENT,4320 HOEPKER,D PORT EQUIPMENT 4320 HOEPKER,USA WI MADISON 4320 HOEPKER,TRAVIS GREGORY DAVENPORT D PORT EQUIPMENT USA...,TRAVIS GREGORY DAVENPORT D PORT EQUIPMENT,1
314422,63555.0,STEVEN HARLEY SANTTI,9.062017e+09,STEVEN HARLEY SANTTI,457 Harrison street,NORTH FOND DU LAC,WI,USA,STEVEN HARLEY SANTTI,457 HARRISON,STEVEN HARLEY SANTTI 457 HARRISON,USA WI NORTH FOND DU LAC 457 HARRISON,STEVEN HARLEY SANTTI STEVEN HARLEY SANTTI USA ...,STEVEN HARLEY SANTTI STEVEN HARLEY SANTTI,1
314423,624599.0,MOHAMMED ADNAN HAMEED,4.145738e+09,MOHAMMED ADNAN HAMEED,419 Southtown pl,SOUTH MILWAUKEE,WI,USA,MOHAMMED ADNAN HAMEED,419 SOUTHTOWN PL,MOHAMMED ADNAN HAMEED 419 SOUTHTOWN PL,USA WI SOUTH MILWAUKEE 419 SOUTHTOWN PL,MOHAMMED ADNAN HAMEED MOHAMMED ADNAN HAMEED US...,MOHAMMED ADNAN HAMEED MOHAMMED ADNAN HAMEED,1
314424,167228.0,CHARLES KOEHN,3.129274e+09,CHARLES KOEHN,7400 N Bridge Ln,MILWAUKEE,WI,USA,CHARLES KOEHN,7400 N BRIDGE LN,CHARLES KOEHN 7400 N BRIDGE LN,USA WI MILWAUKEE 7400 N BRIDGE LN,CHARLES KOEHN CHARLES KOEHN USA WI MILWAUKEE 7...,CHARLES KOEHN CHARLES KOEHN,1
314425,414607.0,WILLIAM KIPPLEY,6.082207e+09,"BCK INVESTORS, INC.",7646 CONIFER CT,DEFOREST,WI,USA,"BCK INVESTORS, .",7646 CONIFER CT,"BCK INVESTORS, . 7646 CONIFER CT",USA WI DEFOREST 7646 CONIFER CT,"WILLIAM KIPPLEY BCK INVESTORS, . USA WI DEFORE...","WILLIAM KIPPLEY BCK INVESTORS, .",1


,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,...,LastLoginDate,LastActivityDate,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,1,GREAT WEST CASUALTY COMPNAY,4.024942e+09,Auction,Administrator,AUCTION ADMINISTRATOR,GW-AuctionAdmin,GW-AuctionAdmin@gwccnet.com,1100 W 29th St,NaN,...,2024-06-28 16:19:09,2024-06-28 16:19:09,USA,GREAT WEST CASUALTY COMPNAY,1100 W 29TH,GREAT WEST CASUALTY COMPNAY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMPNAY,0
1,162383,GREAT WEST CASUALTY,4.024948e+09,Rich,Vandenberg,RICH VANDENBERG,r.vandenberg,r.vandenberg@gwccnet.com,1100 W 29th St,NaN,...,2024-12-18 20:20:52,2024-12-18 20:20:52,USA,GREAT WEST CASUALTY,1100 W 29TH,GREAT WEST CASUALTY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,RICH VANDENBERG GREAT WEST CASUALTY USA NE SOU...,RICH VANDENBERG GREAT WEST CASUALTY,0
2,166344,GREAT WEST CASUALTY,8.173853e+09,Gary,Rheinlander,GARY RHEINLANDER,g.rheinlander,g.rheinlander@gwccnet.com,417 Rolling Hills Trl,NaN,...,2021-04-28 12:31:41,2021-04-28 12:31:41,USA,GREAT WEST CASUALTY,417 ROLLING HILLS TRL,GREAT WEST CASUALTY 417 ROLLING HILLS TRL,USA TX ARLINGTON 417 ROLLING HILLS TRL,GARY RHEINLANDER GREAT WEST CASUALTY USA TX AR...,GARY RHEINLANDER GREAT WEST CASUALTY,0
3,166444,GREAT WEST CASUALTY,4.024943e+09,Doug,Brown,DOUG BROWN,d.brown,d.brown@gwccnet.com,624 Six Flags Dr Ste 240,NaN,...,2024-02-20 15:45:57,2024-02-20 15:45:58,USA,GREAT WEST CASUALTY,624 SIX FLAGS E 240,GREAT WEST CASUALTY 624 SIX FLAGS E 240,USA TX ARLINGTON 624 SIX FLAGS E 240,DOUG BROWN GREAT WEST CASUALTY USA TX ARLINGTO...,DOUG BROWN GREAT WEST CASUALTY,0
4,167044,GREAT WEST CASUALTY,8.002289e+09,Randy,Phillips,RANDY PHILLIPS,r.phillips,r.phillips@gwccnet.com,10100 Global Way Ste 100,NaN,...,2024-12-18 13:51:29,2024-12-18 13:51:29,USA,GREAT WEST CASUALTY,10100 GLOBAL WAY E 100,GREAT WEST CASUALTY 10100 GLOBAL WAY E 100,USA TN KNOXVILLE 10100 GLOBAL WAY E 100,RANDY PHILLIPS GREAT WEST CASUALTY USA TN KNOX...,RANDY PHILLIPS GREAT WEST CASUALTY,0


,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,...,LastLoginDate,LastActivityDate,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
1346,47327521,GWCC,4.024947e+09,Stacey,Oerman,STACEY OERMAN,msoerman,msoerman@yahoo.com,PO BOX 94,NaN,...,2024-12-18 21:43:38,2024-12-18 21:43:39,USA,GWCC,PO BOX 94,GWCC PO BOX 94,USA NE SOUTH SIOUX CITY PO BOX 94,STACEY OERMAN GWCC USA NE SOUTH SIOUX CITY PO ...,STACEY OERMAN GWCC,0
1347,47340076,GREAT WEST CASUALTY,NaN,Daniela,Garcia,DANIELA GARCIA,dgarci3,d.garcia3@gwccnet.com,1100 W 29th ST,NaN,...,2024-12-17 14:52:18,2024-12-17 14:52:18,USA,GREAT WEST CASUALTY,1100 W 29TH,GREAT WEST CASUALTY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,DANIELA GARCIA GREAT WEST CASUALTY USA NE SOUT...,DANIELA GARCIA GREAT WEST CASUALTY,0
1348,47340117,GREAT WEST CASUALTY CO,8.123374e+09,Roberta,Eller,ROBERTA ELLER,R.Eller,r.eller@gwccnet.com,3042 North Lintel Drive,NaN,...,2024-12-16 14:51:57,2024-12-16 14:56:29,USA,GREAT WEST CASUALTY CO,3042 NORTH LINTEL,GREAT WEST CASUALTY CO 3042 NORTH LINTEL,USA IN BLOOMINGTON 3042 NORTH LINTEL,ROBERTA ELLER GREAT WEST CASUALTY CO USA IN BL...,ROBERTA ELLER GREAT WEST CASUALTY CO,0
1349,47347325,GREAT WEST CASUALTY COMPANY,NaN,Brenda,Cohen,BRENDA COHEN,BSCPDC,B.COHEN@GWCCNET.COM,3042 N Lintel Dr,NaN,...,2024-12-18 16:17:30,2024-12-18 16:17:30,USA,GREAT WEST CASUALTY COMPANY,3042 N LINTEL,GREAT WEST CASUALTY COMPANY 3042 N LINTEL,USA IN BLOOOMINGTON 3042 N LINTEL,BRENDA COHEN GREAT WEST CASUALTY COMPANY USA I...,BRENDA COHEN GREAT WEST CASUALTY COMPANY,0
1350,47347876,GREAT WEST CASUALTY COMPANY,8.123374e+09,Jackie,Branaman,JACKIE BRANAMAN,j.branaman@gwccnet.com,j.branaman@gwccnet.com,3042 N. LINTEL DRIVE,NaN,...,2024-12-18 16:09:00,2024-12-18 16:09:00,USA,GREAT WEST CASUALTY COMPANY,3042 N. LINTEL,GREAT WEST CASUALTY COMPANY 3042 N. LINTEL,USA IN BLOOMINGTON 3042 N. LINTEL,JACKIE BRANAMAN GREAT WEST CASUALTY COMPANY US...,JACKIE BRANAMAN GREAT WEST CASUALTY COMPANY,0


In [ ]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('Dave Request.csv')

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
